In [2]:
import pandas as pd
import dask.dataframe as dd
import pyarrow as pa
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder

In [3]:
# Import the data as a dask dataframe
#data = dd.read_csv('/Users/stefanfeiler/Desktop/Golden_nba_dataset.csv')

In [4]:
# Test to see how long it takes to read as a pd df
data = pd.read_csv('/Users/stefanfeiler/Desktop/Projects/+EV HF/golden data set insights/golden_set_odds_probs_times.csv')

/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_61322/3756605212.py:2: DtypeWarning: Columns (22,37,62,67,92) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/stefanfeiler/Desktop/Projects/+EV HF/golden data set insights/golden_set_odds_probs_times.csv')


In [6]:
data.head()

,my_id,game_id,commence_time_est,time_pulled_est,team_1,team_2,winning_team,barstool_last_update_time,barstool_team_1_odds,barstool_team_1_prob,...,williamhillus_last_update_time,williamhillus_team_1_odds,williamhillus_team_1_prob,williamhillus_team_2_odds,williamhillus_team_2_prob,wynnbet_last_update_time,wynnbet_team_1_odds,wynnbet_team_1_prob,wynnbet_team_2_odds,wynnbet_team_2_prob
0,Golden State WarriorsPhoenix Suns12252021,f2f919ab381d9d35957b4bceae34eb10,12/25/21 17:10,12/25/21 16:09,Golden State Warriors,Phoenix Suns,Golden State Warriors,12/25/21 16:00,195.0,0.33,...,12/25/21 16:00,205.0,0.314607,-250.0,0.685393,NaN,NaN,NaN,NaN,NaN
1,Dallas MavericksUtah Jazz12252021,d2cf3ea1305b1f6126054d61ae335b0f,12/25/21 22:40,12/25/21 16:09,Dallas Mavericks,Utah Jazz,Utah Jazz,12/25/21 16:00,700.0,0.12,...,12/25/21 16:00,700.0,0.119913,-1111.0,0.880087,NaN,NaN,NaN,NaN,NaN
2,Brooklyn NetsLos Angeles Lakers12252021,a8ab19e4b76925610ac9e67d43324ca2,12/25/21 20:10,12/25/21 16:09,Brooklyn Nets,Los Angeles Lakers,Brooklyn Nets,12/25/21 16:00,107.0,0.46,...,12/25/21 16:00,110.0,0.457256,-130.0,0.542744,NaN,NaN,NaN,NaN,NaN
3,Boston CelticsMilwaukee Bucks12252021,f710b144d576f85b196af94a143c40fb,12/25/21 14:40,12/25/21 16:09,Boston Celtics,Milwaukee Bucks,Milwaukee Bucks,12/25/21 16:00,-250.0,0.68,...,12/25/21 16:00,-222.0,0.658754,180.0,0.341246,NaN,NaN,NaN,NaN,NaN
4,Boston CelticsMilwaukee Bucks12252021,f710b144d576f85b196af94a143c40fb,12/25/21 14:40,12/25/21 16:19,Boston Celtics,Milwaukee Bucks,Milwaukee Bucks,12/25/21 16:10,-143.0,0.56,...,12/25/21 16:10,-161.0,0.586567,130.0,0.413433,NaN,NaN,NaN,NaN,NaN


In [7]:
# Read the column names 
all_columns = data.columns
columns = all_columns[7:]
odds_columns = [x for x in columns if x.endswith('_1') or x.endswith('_2') or x.endswith('odds')]
time_columns = [x for x in columns if x.endswith('_time')]
prob_columns = [x for x in columns if x.endswith('_prob')]

In [8]:
# See the length of the df
len(data)

182460

In [9]:
data.head()

,my_id,game_id,commence_time_est,time_pulled_est,team_1,team_2,winning_team,barstool_last_update_time,barstool_team_1_odds,barstool_team_1_prob,...,williamhillus_last_update_time,williamhillus_team_1_odds,williamhillus_team_1_prob,williamhillus_team_2_odds,williamhillus_team_2_prob,wynnbet_last_update_time,wynnbet_team_1_odds,wynnbet_team_1_prob,wynnbet_team_2_odds,wynnbet_team_2_prob
0,Golden State WarriorsPhoenix Suns12252021,f2f919ab381d9d35957b4bceae34eb10,12/25/21 17:10,12/25/21 16:09,Golden State Warriors,Phoenix Suns,Golden State Warriors,12/25/21 16:00,195.0,0.33,...,12/25/21 16:00,205.0,0.314607,-250.0,0.685393,NaN,NaN,NaN,NaN,NaN
1,Dallas MavericksUtah Jazz12252021,d2cf3ea1305b1f6126054d61ae335b0f,12/25/21 22:40,12/25/21 16:09,Dallas Mavericks,Utah Jazz,Utah Jazz,12/25/21 16:00,700.0,0.12,...,12/25/21 16:00,700.0,0.119913,-1111.0,0.880087,NaN,NaN,NaN,NaN,NaN
2,Brooklyn NetsLos Angeles Lakers12252021,a8ab19e4b76925610ac9e67d43324ca2,12/25/21 20:10,12/25/21 16:09,Brooklyn Nets,Los Angeles Lakers,Brooklyn Nets,12/25/21 16:00,107.0,0.46,...,12/25/21 16:00,110.0,0.457256,-130.0,0.542744,NaN,NaN,NaN,NaN,NaN
3,Boston CelticsMilwaukee Bucks12252021,f710b144d576f85b196af94a143c40fb,12/25/21 14:40,12/25/21 16:09,Boston Celtics,Milwaukee Bucks,Milwaukee Bucks,12/25/21 16:00,-250.0,0.68,...,12/25/21 16:00,-222.0,0.658754,180.0,0.341246,NaN,NaN,NaN,NaN,NaN
4,Boston CelticsMilwaukee Bucks12252021,f710b144d576f85b196af94a143c40fb,12/25/21 14:40,12/25/21 16:19,Boston Celtics,Milwaukee Bucks,Milwaukee Bucks,12/25/21 16:10,-143.0,0.56,...,12/25/21 16:10,-161.0,0.586567,130.0,0.413433,NaN,NaN,NaN,NaN,NaN


In [10]:
# Replace all the missing odds values with 0's
for col in odds_columns:
    data[col] = data[col].replace(np.nan, 0)
    data[col] = data[col].astype('int32')

In [11]:
# Replace all the missing time values with Jan 1, 1970
for col in time_columns:
    data[col] = data[col].replace(np.nan, '1/1/1970 00:00:00')
    num_zeros = (data[col] == 0).sum()

In [12]:
for col in prob_columns:
    data[col] = data[col].replace(np.nan, 0)

In [13]:
data.head()

,my_id,game_id,commence_time_est,time_pulled_est,team_1,team_2,winning_team,barstool_last_update_time,barstool_team_1_odds,barstool_team_1_prob,...,williamhillus_last_update_time,williamhillus_team_1_odds,williamhillus_team_1_prob,williamhillus_team_2_odds,williamhillus_team_2_prob,wynnbet_last_update_time,wynnbet_team_1_odds,wynnbet_team_1_prob,wynnbet_team_2_odds,wynnbet_team_2_prob
0,Golden State WarriorsPhoenix Suns12252021,f2f919ab381d9d35957b4bceae34eb10,12/25/21 17:10,12/25/21 16:09,Golden State Warriors,Phoenix Suns,Golden State Warriors,12/25/21 16:00,195,0.33,...,12/25/21 16:00,205,0.314607,-250,0.685393,1/1/1970 00:00:00,0,0.0,0,0.0
1,Dallas MavericksUtah Jazz12252021,d2cf3ea1305b1f6126054d61ae335b0f,12/25/21 22:40,12/25/21 16:09,Dallas Mavericks,Utah Jazz,Utah Jazz,12/25/21 16:00,700,0.12,...,12/25/21 16:00,700,0.119913,-1111,0.880087,1/1/1970 00:00:00,0,0.0,0,0.0
2,Brooklyn NetsLos Angeles Lakers12252021,a8ab19e4b76925610ac9e67d43324ca2,12/25/21 20:10,12/25/21 16:09,Brooklyn Nets,Los Angeles Lakers,Brooklyn Nets,12/25/21 16:00,107,0.46,...,12/25/21 16:00,110,0.457256,-130,0.542744,1/1/1970 00:00:00,0,0.0,0,0.0
3,Boston CelticsMilwaukee Bucks12252021,f710b144d576f85b196af94a143c40fb,12/25/21 14:40,12/25/21 16:09,Boston Celtics,Milwaukee Bucks,Milwaukee Bucks,12/25/21 16:00,-250,0.68,...,12/25/21 16:00,-222,0.658754,180,0.341246,1/1/1970 00:00:00,0,0.0,0,0.0
4,Boston CelticsMilwaukee Bucks12252021,f710b144d576f85b196af94a143c40fb,12/25/21 14:40,12/25/21 16:19,Boston Celtics,Milwaukee Bucks,Milwaukee Bucks,12/25/21 16:10,-143,0.56,...,12/25/21 16:10,-161,0.586567,130,0.413433,1/1/1970 00:00:00,0,0.0,0,0.0


In [14]:
# See the amount of missing values in a few columns
percent_list = []
for col in odds_columns:
    zero_count = data[col].value_counts()[0]
    #print(f"{col}: {(zero_count/len(data_pd))*100}")
    percent_list.append((zero_count/len(data))*100)
print(np.array(percent_list).mean())

percent_list = []
for col in time_columns:
    zero_count = data[col].value_counts()[0]
  #print(f"{col}: {(zero_count/len(data_pd))*100}")
    percent_list.append((zero_count/len(data))*100)
print(np.array(percent_list).mean())

percent_list = []
for col in prob_columns:
    zero_count = data[col].value_counts()[0]
    #print(f"{col}: {(zero_count/len(data_pd))*100}")
    percent_list.append((zero_count/len(data))*100)
print(np.array(percent_list).mean())
# About 20 percent of each column doesn't have odds. This means that for each of our input features, about 20% of any given example will be blank. We should make two new datasets, one that fills those  values with 0 and one that fills it with the market average. For now, lets use the 0 full dataset because we're going to design our net to prefer very low weights so that 0's shouldn't affect the activations all that much
# The other side of that is that I don't want it to find relationships when people just dont have odds posted for some reason. 

20.128582215889026
20.128582215889022
20.128582215889026


In [15]:
# Check length of Time since last update for each odds update 
# First we need to convert each time column as a pd.datetime
# With the missing values, we need to see how this time variable will be used further down the road. If we're planning on turning the last update time into a feature that measures difference between that time and the time_pulled or time_since_game_start variable, then we might want that final variable to be negative if there's no value so that the activation never turns on for that feature node in those instances. In that case, we should replace missing vaues with a time value way in the past. So lets do that. 
#time_cols = [x for x in columns if x.endswith('_time')]
for col in time_columns:
    data[col] = pd.to_datetime(data[col])

In [16]:
# We need to make the 'snapshot_time' variable which is the max value of the time columns by row

# find max value in each row and set it as the value of 'snapshot_time' column
data['snapshot_time_taken'] = data[time_columns].apply(lambda x: max(x), axis=1)

In [17]:
# We need to use that column as the time_pulled variable rather than the actual time pulled 
data['snapshot_time_taken'] = pd.to_datetime(data['snapshot_time_taken'])
data['commence_time_est'] = pd.to_datetime(data['commence_time_est'])

In [18]:
data['minutes_since_commence'] = (data['snapshot_time_taken'] - data['commence_time_est']).dt.total_seconds()/60

In [19]:
data['hour_of_start'] = data['commence_time_est'].dt.hour

In [20]:
data['day_of_week'] = data['commence_time_est'].dt.weekday

In [21]:
# can always manipulate the data to get more here......

In [22]:
# Only look at data where the snapshot time is less than 6 hours from the start of the game 
new_data = data[data['minutes_since_commence'] >= -360]

In [23]:
new_data['game_id']

0         f2f919ab381d9d35957b4bceae34eb10
2         a8ab19e4b76925610ac9e67d43324ca2
3         f710b144d576f85b196af94a143c40fb
4         f710b144d576f85b196af94a143c40fb
5         f2f919ab381d9d35957b4bceae34eb10
                        ...               
182080    160a4534193f842917b69c0028d2eb16
182081    8cc33d033fd99025a3c362e33c846641
182087    160a4534193f842917b69c0028d2eb16
182088    8cc33d033fd99025a3c362e33c846641
182094    8cc33d033fd99025a3c362e33c846641
Name: game_id, Length: 43781, dtype: object

In [24]:
# Now that we have all of our variables, we're going to stack each section of the df

# Select all the columns that contain a '1' and '2'
cols_with_one = [col for col in new_data.columns if '1' in col and not 'odds' in col]
cols_with_two = [col for col in new_data.columns if '2' in col and not 'odds' in col]

# Select all the last_update columns and make it into a new df
last_update_columns = [col for col in new_data.columns if 'last_update_time' in col]
df_update_cols = new_data[last_update_columns]

df_with_one = new_data[cols_with_one]
df_with_two = new_data[cols_with_two]

# Add the game_id column 
df_with_one['game_id'] = new_data['game_id']
df_with_two['game_id'] = new_data['game_id']

# Add the winning team column 
df_with_one['winning_team'] = new_data['winning_team']
df_with_two['winning_team'] = new_data['winning_team']

# Add the minutes_since_commence column
df_with_one['minutes_since_commence'] = new_data['minutes_since_commence']
df_with_two['minutes_since_commence'] = new_data['minutes_since_commence']

# Add the snapshot_time_taken column
df_with_one['snapshot_time_taken'] = new_data['snapshot_time_taken']
df_with_two['snapshot_time_taken'] = new_data['snapshot_time_taken']

# Add the hour_of_start column
df_with_one['hour_of_start'] = new_data['hour_of_start']
df_with_two['hour_of_start'] = new_data['hour_of_start']

# Add the day_of_week column
df_with_one['day_of_week'] = new_data['day_of_week']
df_with_two['day_of_week'] = new_data['day_of_week']

# Add the home/away variable
df_with_one['home_away'] = 0
df_with_two['home_away'] = 1
    
# Add the last update time columnssss
df_1 = pd.concat([df_with_one, df_update_cols], axis=1)
df_2 = pd.concat([df_with_two, df_update_cols], axis=1)


# Rename 
# Concat

# # Get list of column names from df1
df1_cols = df_1.columns.tolist()

# # Create dictionary to map column names in df2 to column names in df1
col_map = {col: df1_cols[i] for i, col in enumerate(df_2.columns)}

# # Rename columns in df2 using dictionary
df_2 = df_2.rename(columns=col_map)

# # Concatenate vertically
result = pd.concat([df_1, df_2], ignore_index=True, axis=0)

# Add the first potential target variable
result['target'] = np.where(result['team_1'] == result['winning_team'], 1, 0)

# We might need to clean this up to only contain odds that are within 5 seconds of the snapshot time, or the best odds within 5 seconds etc.


/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_61322/2013073024.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_one['game_id'] = new_data['game_id']
/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_61322/2013073024.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_two['game_id'] = new_data['game_id']
/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_61322/2013073024.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [25]:
result.columns

Index(['team_1', 'barstool_team_1_prob', 'betfair_team_1_prob',
       'betmgm_team_1_prob', 'betonlineag_team_1_prob',
       'betrivers_team_1_prob', 'bovada_team_1_prob',
       'circasports_team_1_prob', 'draftkings_team_1_prob',
       'fanduel_team_1_prob', 'foxbet_team_1_prob', 'gtbets_team_1_prob',
       'pinnacle_team_1_prob', 'pointsbetus_team_1_prob',
       'sugarhouse_team_1_prob', 'twinspires_team_1_prob',
       'unibet_team_1_prob', 'williamhillus_team_1_prob',
       'wynnbet_team_1_prob', 'game_id', 'winning_team',
       'minutes_since_commence', 'snapshot_time_taken', 'hour_of_start',
       'day_of_week', 'home_away', 'barstool_last_update_time',
       'betfair_last_update_time', 'betmgm_last_update_time',
       'betonlineag_last_update_time', 'betrivers_last_update_time',
       'bovada_last_update_time', 'circasports_last_update_time',
       'draftkings_last_update_time', 'fanduel_last_update_time',
       'foxbet_last_update_time', 'gtbets_last_update_time',

In [26]:
# Standardize all the other variables 

In [30]:
# All done for now
result.to_csv('/Users/stefanfeiler/Desktop/result.csv', index=False)

In [29]:
result['home_away'].tail()

87557    1
87558    1
87559    1
87560    1
87561    1
Name: home_away, dtype: int64

In [31]:
params_count = (65*800)+(800*800)+(800*800)+(800*800)+(800*256)+256
params_count

2177056